In [29]:
import requests
import zipfile
import io

# Zugangsdaten einfügen
USERNAME = "einfügen..."
PASSWORD = "einfügen..."
TABLE = "Wanderungsstatistik"
TABLE_CODE = "12711-0005"


In [31]:
# API-Endpunkt
url = "https://www-genesis.destatis.de/genesisWS/rest/2020/data/tablefile"

# Anfrageparameter
params = {
    "username": USERNAME,
    "password": PASSWORD,
    "name": TABLE_CODE,
    "area": "all",
    "compress": "false",      # falls ZIP-Datei auf true setzen
    "transpose": "false",
    "startyear": "2015",
    "endyear": "2023",
    "language": "de"
}

# API-Anfrage
response = requests.get(url, params=params)

# Erfolg prüfen
if response.status_code == 200:
    content_type = response.headers.get("Content-Type", "")
    print("📄 Content-Type:", content_type)

    if "zip" in content_type or params["compress"] == "true":
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                print("📦 Enthaltene Dateien:", z.namelist())
                extracted_file = z.namelist()[0]
                z.extract(extracted_file, path=".")
                print(f"✅ Datei erfolgreich extrahiert: {extracted_file}")

                 # Neuen Namen definieren
                new_filename = f"{TABLE}_{TABLE_CODE}.csv"
    
                # Ursprüngliche Datei umbenennen
                os.rename(extracted_file, new_filename)
                print(f"✅ Datei gespeichert als: {new_filename}")
        except zipfile.BadZipFile:
            print("❌ Fehler: Datei ist keine gültige ZIP-Datei.")
    else:
        # Nicht komprimierte Datei speichern
        with open(f"{TABLE}_{TABLE_CODE}.csv", "wb") as f:
            f.write(response.content)
        print(f"✅ CSV-Datei gespeichert als {TABLE}_{TABLE_CODE}.csv")
else:
    print("❌ Fehler beim Abrufen der Daten.")
    print("Status Code:", response.status_code)
    print("Antwort:", response.text)

📄 Content-Type: application/zip
📦 Enthaltene Dateien: ['12711-0005_1746622722680_de.csv']
✅ Datei erfolgreich extrahiert: 12711-0005_1746622722680_de.csv
✅ Datei gespeichert als: Wanderungsstatistik_12711-0005.csv
